# AUXILIARY FUNCTIONS

In [94]:
import numpy as np
import matplotlib.pyplot as plt
from string import upper
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage

# FUNCTION RLDESC_C

def rl(f, nc, cc):
    result = np.zeros((f.max()+1,max(np.unique(cc))), dtype = np.int32)
    for i in range (0, nc.size):
        result[nc[i],cc[i]-1]+=1
    return result
        

# FUNCTIONS IA636

def iah2percentile(h, p):
    s = h.sum()
    k = ((s-1) * p/100.)+1
    dw = np.floor(k)
    up = np.ceil(k)
    hc = np.cumsum(h)
    if isinstance(p, int):
        k1 = np.argmax(hc>=dw)
        k2 = np.argmax(hc>=up)
    else:
        k1 = np.argmax(hc>=dw[:, np.newaxis], axis=1)
        k2 = np.argmax(hc>=up[:, np.newaxis], axis=1)
    d0 = k1 * (up-k)
    d1 = k2 * (k-dw)
    
    return np.where(dw==up, k1, d0+d1)

def iaconv(f, h):
    f, h = np.asarray(f), np.asarray(h,float)
    if len(f.shape) == 1: f = f[np.newaxis,:]
    if len(h.shape) == 1: h = h[np.newaxis,:]
    if f.size < h.size:
        f, h = h, f
    g = np.zeros(np.array(f.shape) + np.array(h.shape) - 1)
    if f.ndim == 2:
        H,W = f.shape
        for (r,c) in np.transpose(np.nonzero(h)):
            g[r:r+H, c:c+W] += f * h[r,c]

    if f.ndim == 3:
        D,H,W = f.shape
        for (d,r,c) in np.transpose(np.nonzero(h)):
            g[d:d+D, r:r+H, c:c+W] += f * h[d,r,c]

    return g


def ianormalize(f, range=[0,255]):
        
    f = np.asarray(f)
    range = np.asarray(range)
    if f.dtype.char in ['D', 'F']:
        raise Exception, 'error: cannot normalize complex data'
    faux = np.ravel(f).astype(float)
    minimum = faux.min()
    maximum = faux.max()
    lower = range[0]
    upper = range[1]
    if upper == lower:
        g = np.ones(f.shape) * maximum
    if minimum == maximum:
        g = np.ones(f.shape) * (upper + lower) / 2.
    else:
        g = (faux-minimum)*(upper-lower) / (maximum-minimum) + lower
    g = np.reshape(g, f.shape)
        
    if f.dtype == np.uint8:
        if upper > 255: 
            raise Exception,'ianormalize: warning, upper valuer larger than 255. Cannot fit in uint8 image'
    g = g.astype(f.dtype) # set data type of result the same as the input image
    return g

def iacontour(f):
    f_ = f > 0
    new_shape = np.array(f_.shape)+2
    n = np.zeros(new_shape); n[0:-2,1:-1] = f_
    s = np.zeros(new_shape); s[2:: ,1:-1] = f_
    w = np.zeros(new_shape); w[1:-1,0:-2] = f_
    e = np.zeros(new_shape); e[1:-1,2:: ] = f_
    fi = np.logical_and(np.logical_and(np.logical_and(n,s),w),e)
    fi = fi[1:-1,1:-1]
    g = f_.astype(int) - fi.astype(int)
    g = g > 0
    return g

    
# FUNCTIONS FROM IA870

def iaadd4dil(f, c):

    if not c:
        return f
    if f.dtype == 'float64':
        y = f + c
    else:
        y = np.asarray(f,np.int64) + c
        k1,k2 = ialimits(f)
        y = ((f==k1) * k1) + ((f!=k1) * y)
        y = np.clip(y,k1,k2)
    a = y.astype(f.dtype)
    return a

def iabinary(f, k1=1):
    f = np.asarray(f)
    y = f >= k1            
    return y

def iadil(f, b=None):

    if b is None: b = iasecross()
        
    if len(f.shape) == 1: f = f[np.newaxis,:]
    h,w = f.shape
    x,v = iamat2set(b)
    if len(x)==0:
        y = (np.ones((h,w)) * ialimits(f)[0]).astype(f.dtype)
    else:
        if iaisbinary(v):
            v = iaintersec( iagray(v,'int32'),0)
        mh,mw = max(abs(x)[:,0]),max(abs(x)[:,1])
        y = (np.ones((h+2*mh,w+2*mw)) * ialimits(f)[0]).astype(f.dtype)
        for i in range(x.shape[0]):
            if v[i] > -2147483647:
                y[mh+x[i,0]:mh+x[i,0]+h, mw+x[i,1]:mw+x[i,1]+w] = np.maximum(
                    y[mh+x[i,0]:mh+x[i,0]+h, mw+x[i,1]:mw+x[i,1]+w], iaadd4dil(f,v[i]))
        y = y[mh:mh+h, mw:mw+w]
            
    return y

def iagray(f, TYPE="uint8", k1=None):
    ff = np.array([0],TYPE)
    kk1,kk2 = ialimits(ff)
    if k1!=None:
        kk2=k1
    if   TYPE == 'uint8'  : y = np.where(f,kk2,kk1).astype(np.uint8)
    elif TYPE == 'uint16' : y = np.where(f,kk2,kk1).astype(np.uint16)
    elif TYPE == 'int32'  : y = np.where(f,kk2,kk1).astype(np.int32)
    elif TYPE == 'int64'  : y = np.where(f,kk2,kk1).astype(np.int64)
    elif TYPE == 'float64': y = np.where(f,kk2,kk1).astype(np.float64)
    else:
        assert 0, 'type not supported:'+TYPE
    return y

def ianeg(f):

    if ialimits(f)[0] == (- ialimits(f)[1]):
        y = -f
    else:
        y = ialimits(f)[0] + ialimits(f)[1] - f
    y = y.astype(f.dtype)     
    return y

def iaero(f, b=None):

    if b is None: b = iasecross()
    y = ianeg( iadil( ianeg(f),iasereflect(b)))
    return y

def iaisbinary(f):
        
    return f.dtype == bool

def ialimits(f):
    code = f.dtype
    if   code == bool:   y=np.array([0,1],'bool')
    elif code == np.uint8:  y=np.array([0,255],'uint8')
    elif code == np.uint16: y=np.array([0,(2**16)-1],'uint16')
    elif code == np.int32:  y=np.array([-((2**31)-1),(2**31)-1],'int32')
    elif code == np.int64:  y=np.array([-((2**63)-1), (2**63)-1],'int64')
    elif code == np.float64:y=np.array([-np.inf,np.inf],'float64')
    else:
        assert 0,'ialimits: Does not accept this typecode:%s' % code
    return y

def iamat2set(A):

    if len(A.shape) == 1: A = A[np.newaxis,:]
    offsets = np.nonzero(np.ravel(A).astype(int)-ialimits(A).astype(int)[0])
    if type(offsets) == type(()):
        offsets = offsets[0]        # for compatibility with numarray
    if len(offsets) == 0: return ([],[])
    (h,w) = A.shape
    x = range(2)
    x[0] = offsets/w - (h-1)/2
    x[1] = offsets%w - (w-1)/2
    x = np.transpose(x)
    CV = x,np.ravel(A)[offsets]           
    return CV

def iaset2mat(A):
    if len(A) == 2:
        x, v = A
        v = np.asarray(v)
    elif len(A) == 1:
        x = A[0]
        v = np.ones((len(x)), bool)
    else:
        raise TypeError, 'Argument must be a tuple of length 1 or 2'
    if len(x) == 0: 
        return np.array([0]).astype(v.dtype)
    if len(x.shape) == 1: 
        x = x[np.newaxis, :]
    dh, dw = np.abs(x).max(0)
    h= (2*dh) + 1
    w = (2*dw) + 1
    M = np.ones((h, w)) * ialimits(v)[0]
    offset = x[:,0] * w + x[:,1] + (dh*w + dw)
    M.flat[offset] = v
    M = M.astype(v.dtype)
    
    return M

def iaintersec(f1, f2, *args):
        
    y = np.minimum(f1,f2)
    for f in args:
        y = np.minimum(y,f)
    return y.astype(f1.dtype)

def iase2off(Bc,option='neigh'):
    

    if(Bc.ndim == 2):
        h, w = Bc.shape
        hc,wc = h/2,w/2
        B = Bc.copy()
        B[hc,wc] = 0  # remove origin
        off = np.transpose(B.nonzero()) - np.array([hc,wc])
    if(Bc.ndim == 3):
        h, w, d = Bc.shape
        hc, wc, dc = h/2, w/2, d/2
        B = Bc.copy()
        B[hc, wc, dc] = 0 # remove origin
        off = np.transpose(B.nonzero()) - np.array([hc,wc, dc])
    
    
    if option == 'neigh':
        return off  # 2 columns x n. of neighbors rows
    elif option == 'fw':
        i = off[:,0] * w + off[:,1] 
        return off[i>0,:]  # only neighbors higher than origin in raster order
    elif option == 'bw':
        i = off[:,0] * w + off[:,1] 
        return off[i<0,:]  # only neighbors less than origin in raster order
    else:
        assert 0,'options are neigh, fw or bw. It was %s'% option
        return None
  

def iasedil(B1, B2):
        
    assert (iaisbinary(B1) or (B1.dtype == np.int32) or (B1.dtype == np.int64) or (B1.dtype == np.float64)) and \
               (iaisbinary(B2) or (B2.dtype == np.int32) or (B2.dtype == np.int64) or (B2.dtype == np.float64)), \
               'iasedil: s.e. must be binary, int32, int64 or float64'
    if len(B1.shape) == 1: B1 = B1[np.newaxis,:]
    if len(B2.shape) == 1: B2 = B2[np.newaxis,:]
    if B1.dtype=='bool' and B2.dtype == 'bool':
        Bo = iabinary([0])
    else:
        Bo = np.array(ialimits(B1)[0]).reshape(1)
        if iaisbinary(B1):
            Bo = np.array(ialimits(B2)[0]).reshape(1)
            B1 = iagray(B1,B2.dtype,0)
        if iaisbinary(B2):
            Bo = array(ialimits(B1)[0]).reshape(1)
            B2 = iagray(B2,B1.dtype,0)
    x,v = iamat2set(B2)
    if len(x):
        for i in range(x.shape[0]):
            s = iaadd4dil(B1,v[i])
            st= iasetrans(s,x[i])
            Bo = iaseunion(Bo,st)
    return Bo

def iasesum(B, N=1): 
    if N==0:
        if iaisbinary(B): return iabinary([1])
        else:             return np.array([0],np.int32) # identity
    NB = B
    for i in range(N-1):
        NB = iasedil(NB,B)
    return NB    
    
def iaseunion(B1, B2):
    
    if B1.dtype != B2.dtype:
        print 'B1=',B1
        print 'B2=',B2
    assert B1.dtype == B2.dtype, \
        'iaseunion: Cannot have different datatypes: \
        %s and %s' % (str(B1.dtype), str(B2.dtype))
        
    type1 = B1.dtype
    
    if len(B1.shape) == 1: B1 = B1[np.newaxis,:]
    if len(B2.shape) == 1: B2 = B2[np.newaxis,:]
        
    if B1.shape <> B2.shape:
        inf = ialimits(B1)[0]
        h1,w1 = B1.shape
        h2,w2 = B2.shape
        H,W = max(h1,h2),max(w1,w2)
        Hc,Wc = (H-1)/2,(W-1)/2    # center
        BB1,BB2 = np.asarray(B1),np.asarray(B2)
        B1, B2  = inf * np.ones((H,W)), inf *np.ones((H,W))
        dh1s , dh1e = (h1-1)/2 , (h1-1)/2 + (h1+1)%2 # deal with even and odd dimensions
        dw1s , dw1e = (w1-1)/2 , (w1-1)/2 + (w1+1)%2
        dh2s , dh2e = (h2-1)/2 , (h2-1)/2 + (h2+1)%2
        dw2s , dw2e = (w2-1)/2 , (w2-1)/2 + (w2+1)%2
        B1[ Hc-dh1s : Hc+dh1e+1  ,  Wc-dw1s : Wc+dw1e+1 ] = BB1
        B2[ Hc-dh2s : Hc+dh2e+1  ,  Wc-dw2s : Wc+dw2e+1 ] = BB2
    B = np.maximum(B1,B2).astype(type1)
    return B

def iasetrans(Bi, t):
  
    x,v=iamat2set(Bi)
    Bo = iaset2mat((x+t,v))
    Bo = Bo.astype(Bi.dtype)
                    
    return Bo

def iasecross(r=1):
    B = iasesum( iabinary([[0,1,0],
                           [1,1,1],
                           [0,1,0]]),r)      
    return B


def iaserot(B, theta=45, DIRECTION="CLOCKWISE"):
    
    DIRECTION = upper(DIRECTION)            
    if DIRECTION == "ANTI-CLOCKWISE":
        theta = -theta
    SA = iamat2set(B)
    theta = np.pi * theta/180
    (y,v)=SA
    if len(y)==0: return iabinary([0])
    x0 = y[:,1] * np.cos(theta) - y[:,0] * np.sin(theta)
    x1 = y[:,1] * np.sin(theta) + y[:,0] * np.cos(theta)
    x0 = np.int32((x0 +0.5)*(x0>=0) + (x0-0.5)*(x0<0))
    x1 = np.int32((x1 +0.5)*(x1>=0) + (x1-0.5)*(x1<0))
    x = np.transpose(np.array([np.transpose(x1),np.transpose(x0)]))
    BROT = iaset2mat((x,v))   
    return BROT


def iasereflect(Bi):
    Bo = iaserot(Bi, 180)
    return Bo

def iaNlut(s,offset):

    if len(s) == 2:
        H, W = s
        n = H*W
        hi = np.arange(H).reshape(H,1)
        wi = np.arange(W).reshape(1,W)
        hoff = offset[:,0]
        woff = offset[:,1]
        h = hi + hoff.reshape(-1,1,1)
        w = wi + woff.reshape(-1,1,1)
        h[(h<0) | (h>=H)] = n
        w[(w<0) | (w>=W)] = n

        Nlut = np.clip(h * W + w,0,n)
        print 'Nlut: \n', Nlut
        return Nlut.reshape(offset.shape[0],-1).transpose()
    else: 
        H,W, L = s
        n = H*W*L
        hi = np.arange(H).reshape(H, 1, 1)
        wi = np.arange(W).reshape(1, W, 1)
        li = np.arange(L).reshape(1, 1, L)
        hoff = offset[:,0]
        print 'hoff: ', hoff
        woff = offset[:,1]
        print 'woff: ', woff
        loff = offset[:, 2]
        print 'loff: ', loff
        h = hi + hoff.reshape(-1,1,1)
        w = wi + woff.reshape(-1,1,1)
        l = li + loff.reshape(-1,1,1)
        print 'h: ', h, '\nw: ', w ,'\nl: ', l
        h[(h<0) | (h>=H)] = n
        w[(w<0) | (w>=W)] = n
        l[(l<0) | (l>=L)] = n
        print 'h: ', h, '\nw: ', w ,'\nl: ', l
        print 'h*W:\n', h*W
        print 'w*L:\n', l*H
        Nlut = np.clip(h*W+w*L,0,n)
        return Nlut.reshape(offset.shape[0],-1).transpose()

 
def iasebox(r=1):
    B = iasesum( iabinary([[1,1,1],
                           [1,1,1],
                           [1,1,1]]),r)
    return B

def iasedisk(r=3, DIM="2D", METRIC="EUCLIDEAN", FLAT="FLAT", h=0):

    METRIC = upper(METRIC)
    FLAT   = upper(FLAT)            
    assert DIM=='2D','Supports only 2D structuring elements'
    if FLAT=='FLAT': y = iabinary([1])
    else:            y = int32([h])
    if r==0: return y
    if METRIC == 'EUCLIDEAN':
        v = np.arange(-r,r+1)
        x = np.resize(v, (len(v), len(v)))
        y = np.transpose(x)
        Be = iabinary(np.sqrt(x*x + y*y) <= (r+0.5))
        if FLAT=='FLAT':
            return Be
        be = h + int32(np.sqrt( maximum((r+0.5)*(r+0.5) - (x*x) - (y*y),0)))
        be = iaintersec( iagray(Be,'int32'),be)
        return be
    else:
        assert 0,'Non valid metric'
                    
                
    return B

def ialabelflat(f, Bc=iasecross(), delta=0):
    fr = f.astype(np.int) + 1
    return ialabelflat_unionfind(fr,Bc,delta)
 
# implementation by union find
def Find(x,parents): # uses path compression
    if parents[x] == x:
        return x
    else:
        parents[x] = Find(parents[x],parents)
        return parents[x]

def Union(n, p,parents):
    r = Find(n,parents)
    if r != p:
        parents[r] = p

def ialabelflat_unionfind(img,Bc,delta):
    imshape = img.shape
    imsize  = img.size
    
    # Offsets and neighbors
    offsets = iase2off(Bc,'fw')
    print 'offsets: ', offsets
    neighbors = iaNlut(imshape,offsets)
    parents = np.arange(imsize,dtype = int)
    # Raster image and non-zero pixels
    img = img.ravel()
    nonzero_nodes = np.nonzero(img)[0]

    print 'nonzero_nodes from iabelflat_unionfind', nonzero_nodes
    img =  np.concatenate((img,np.array([0])))
    g = np.zeros(imsize, dtype = int)
    cur_label = 0

    # pass 1: backward scan, forward neighbors
    for p in nonzero_nodes[::-1]:
        v = img[p] 
        for nb in neighbors[p]:
            condition = np.logical_and((img[nb].all()).astype(np.bool), (((abs(v ^ img[nb])) <= delta).all()))
            if condition:
                Union(nb,p,parents)
    # pass 2_1: root labeled
    pbool = parents[nonzero_nodes] == nonzero_nodes
    labels = np.arange(1,pbool.sum()+1)


    g[nonzero_nodes[pbool]] = labels
    # pass 2_2: labeling root descendants
    for p in nonzero_nodes[~pbool]:
        g[p] = g[parents[p]]
    return g.reshape(imshape)

ialimits example

In [95]:
print ialimits(np.array([0, 1, 0],bool))
print ialimits(np.array([0, 1, 2],np.uint8))
print ialimits(np.array([0, 1, 2],np.uint16))
print ialimits(np.array([0, 1, 2],np.int32))
print ialimits(np.array([0, 1, 2],np.int64))


[False  True]
[  0 255]
[    0 65535]
[-2147483647  2147483647]
[-9223372036854775807 9223372036854775807]


iamat2set example 

In [96]:
f= np.array([[1,2,3],
             [4,5,6],
             [7,8,9]])
i,v=iamat2set(f)
print 'f=\n',f
print 'f.dtype=',f.dtype
print 'coordinates=\n',i
print 'v=\n',v
print 'v.dtype=',v.dtype

f=
[[1 2 3]
 [4 5 6]
 [7 8 9]]
f.dtype= int64
coordinates=
[[-1 -1]
 [-1  0]
 [-1  1]
 [ 0 -1]
 [ 0  0]
 [ 0  1]
 [ 1 -1]
 [ 1  0]
 [ 1  1]]
v=
[1 2 3 4 5 6 7 8 9]
v.dtype= int64


iaadd4dill example

In [97]:
f = np.array([-np.inf, -30, 4, 33, -np.inf])
print 'f.dtype', f.dtype
print 'f:',f
print 'f + 3:',iaadd4dil(f,3)
print 'f - 10',iaadd4dil(f,-10)

f.dtype float64
f: [-inf -30.   4.  33. -inf]
f + 3: [-inf -27.   7.  36. -inf]
f - 10 [-inf -40.  -6.  23. -inf]


iagray example 

In [98]:
b=iabinary([0, 1, 0, 1])
print b
c=iagray(b)
print c
d=iagray(b,'uint8',100)
print d
e=iagray(b,'uint16')
print e
f=iagray(b,'int32')
print f
f=iagray(b,'float64')
print f
f=iagray(b,'float64',0)
print f

[False  True False  True]
[  0 255   0 255]
[  0 100   0 100]
[    0 65535     0 65535]
[-2147483647  2147483647 -2147483647  2147483647]
[-inf  inf -inf  inf]
[-inf   0. -inf   0.]


iasedil example 

In [99]:
B1 = iasecross(1)
B2 = iasedil(B1,B1)
print 'B2',B2 *1

f1 = np.array([[-1.,0.,-2.]])
f2 = np.array([[-1.],[0.],[-2.]])
fo = iasedil(f1,f2)
print 'fo', fo

B2 [[0 0 1 0 0]
 [0 1 1 1 0]
 [1 1 1 1 1]
 [0 1 1 1 0]
 [0 0 1 0 0]]
fo [[-2. -1. -3.]
 [-1.  0. -2.]
 [-3. -2. -4.]]


iasecross example 

In [100]:
b1 = iasecross()
print b1
b2 = iasecross(2)
print b2

[[False  True False]
 [ True  True  True]
 [False  True False]]
[[False False  True False False]
 [False  True  True  True False]
 [ True  True  True  True  True]
 [False  True  True  True False]
 [False False  True False False]]


iaseunion example 

In [101]:
f = np.array([-np.inf, -8., 0., 3., np.inf])
g = np.array([0., 1., 2.])
print iaseunion(f,g)


[[-inf   0.   1.   3.  inf]]


iase2off example

In [102]:
B4 = iasecross(1)
print B4
off4 = iase2off(B4)
print 'off4=\n', off4

off4fw = iase2off(B4,'fw')
print 'off4fw=\n', off4fw

off4bw = iase2off(B4, 'bw')
print 'off4bw=\n', off4bw

[[False  True False]
 [ True  True  True]
 [False  True False]]
off4=
[[-1  0]
 [ 0 -1]
 [ 0  1]
 [ 1  0]]
off4fw=
[[0 1]
 [1 0]]
off4bw=
[[-1  0]
 [ 0 -1]]


iaNlut example

In [103]:
B4 = iasecross(1)
print B4
off4 = iase2off(B4)
f = np.zeros((3,4), np.uint8)
Nlut4 = iaNlut(f.shape, off4)
print Nlut4


[[False  True False]
 [ True  True  True]
 [False  True False]]
Nlut: 
[[[12 12 12 12]
  [ 0  1  2  3]
  [ 4  5  6  7]]

 [[12  0  1  2]
  [12  4  5  6]
  [12  8  9 10]]

 [[ 1  2  3 12]
  [ 5  6  7 12]
  [ 9 10 11 12]]

 [[ 4  5  6  7]
  [ 8  9 10 11]
  [12 12 12 12]]]
[[12 12  1  4]
 [12  0  2  5]
 [12  1  3  6]
 [12  2 12  7]
 [ 0 12  5  8]
 [ 1  4  6  9]
 [ 2  5  7 10]
 [ 3  6 12 11]
 [ 4 12  9 12]
 [ 5  8 10 12]
 [ 6  9 11 12]
 [ 7 10 12 12]]


ialabelflat example 

In [104]:
f = np.array([
    [5,5,8,3,0],
    [5,8,8,0,2]], 'uint8')

g = ialabelflat(f)
print 'g=\n', g
g = ialabelflat(f, iasecross(), 2)
print 'g'
print g

offsets:  [[0 1]
 [1 0]]
Nlut: 
[[[ 1  2  3  4 10]
  [ 6  7  8  9 10]]

 [[ 5  6  7  8  9]
  [10 10 10 10 10]]]
nonzero_nodes from iabelflat_unionfind [0 1 2 3 4 5 6 7 8 9]
g=
[[1 1 2 3 4]
 [1 2 2 5 6]]
offsets:  [[0 1]
 [1 0]]
Nlut: 
[[[ 1  2  3  4 10]
  [ 6  7  8  9 10]]

 [[ 5  6  7  8  9]
  [10 10 10 10 10]]]
nonzero_nodes from iabelflat_unionfind [0 1 2 3 4 5 6 7 8 9]
g
[[1 1 2 3 4]
 [1 2 2 4 4]]


In [105]:
from skimage.morphology import label
def iarl(f,theta=0,phi=0):
    
    if len(f.shape)==2: # 2D input
        if theta == np.pi/4: #rl right diagonal
            ee = np.array([[0,0,1],[0,1,0],[1,0,0]]).astype(bool)
        elif theta == np.pi/2: # run lengh vertical
            ee = np.ones((3,1)).astype(bool)
        elif theta == 3*np.pi/4:#rl left diagonal
            ee = np.array([[1,0,0],[0,1,0],[0,0,1]]).astype(bool)
        else:# default option: horizontal run length
            ee = np.ones((1,3)).astype(bool)
    else: # 3D images
        if theta == np.pi/4:
            if phi == np.pi/4: #(45,45)
                ee = np.array([[[0,0,1],[0,0,0],[0,0,0]],[[0,0,0],[0,1,0],[0,0,0]],[[0,0,0],[0,0,0],[1,0,0]]]).astype(bool)
            elif phi == np.pi/2: #(45,90)
                ee = np.array([[[0,0,0],[0,0,0],[0,0,0]],[[0,0,1],[0,1,0],[1,0,0]],[[0,0,0],[0,0,0],[0,0,0]]]).astype(bool)
            elif phi == 3*np.pi/4: #(45,135)
                ee = np.array([[[0,0,0],[0,0,0],[1,0,0]],[[0,0,0],[0,1,0],[0,0,0]],[[0,0,1],[0,0,0],[0,0,0]]]).astype(bool)
        elif theta == np.pi/2:
            if phi == np.pi/4: #(90,45)
                ee = np.array([[[0,0,0],[0,0,0],[0,1,0]],[[0,0,0],[0,1,0],[0,0,0]],[[0,1,0],[0,0,0],[0,0,0]]]).astype(bool)
            elif phi == np.pi/2: #(90,90)
                ee = np.array([[[0,0,0],[0,0,0],[0,0,0]],[[0,1,0],[0,1,0],[0,1,0]],[[0,0,0],[0,0,0],[0,0,0]]]).astype(bool)
            elif phi == 3*np.pi/4: #(90,135)
                ee = np.array([[[0,1,0],[0,0,0],[0,0,0]],[[0,0,0],[0,1,0],[0,0,0]],[[0,0,0],[0,0,0],[0,1,0]]]).astype(bool)
        elif theta == 3*np.pi/4:
            if phi == np.pi/4: #(135,45)
                ee = np.array([[[0,0,0],[0,0,0],[0,0,1]],[[0,0,0],[0,1,0],[0,0,0]],[[1,0,0],[0,0,0],[0,0,0]]]).astype(bool)
            elif phi == np.pi/2: #(135,90)
                ee = np.array([[[0,0,0],[0,0,0],[0,0,0]],[[1,0,0],[0,1,0],[0,0,1]],[[0,0,0],[0,0,0],[0,0,0]]]).astype(bool)
            elif phi == 3*np.pi/4: # (135,135)
                ee = np.array([[[1,0,0],[0,0,0],[0,0,0]],[[0,0,0],[0,1,0],[0,0,0]],[[0,0,0],[0,0,0],[0,0,1]]]).astype(bool)
        else: # default option theta = 0
            if phi == np.pi/4:# (0,45)
                ee = np.array([[[0,0,0],[0,0,1],[0,0,0]],[[0,0,0],[0,1,0],[0,0,0]],[[0,0,0],[1,0,0],[0,0,0]]]).astype(bool)
            elif phi == np.pi/2: # (0,90)
                ee =  np.array([[[0,0,0],[0,0,0],[0,0,0]],[[0,0,0],[1,1,1],[0,0,0]],[[0,0,0],[0,0,0],[0,0,0]]]).astype(bool)
            elif phi == 3*np.pi/4: #(0,135)
                ee =  np.array([[[0,0,0],[1,0,0],[0,0,0]],[[0,0,0],[0,1,0],[0,0,0]],[[0,0,0],[0,0,1],[0,0,0]]]).astype(bool)
            else:# straight up (0,0)
                ee =  np.array([[[0,0,0],[0,1,0],[0,0,0]],[[0,0,0],[0,1,0],[0,0,0]],[[0,0,0],[0,1,0],[0,0,0]]]).astype(bool)
    print 'THIS IS EE.NDIM\n', ee.ndim
    fr = ialabelflat(f,ee)
 
    u, indices = np.unique(np.ravel(fr),return_index=True)
    f = np.ravel(f)
    nc =  f[np.int_(indices)] # check the gray level of each blob
    y = np.bincount(np.ravel(fr))
    ii = np.nonzero(y)[0]
    cc = y[ii] # check the size of each blob
    result = rl(f, nc.astype(np.int32),cc.astype(np.int32))

    return result*1.0


In [106]:
def rldesc(f,theta=0,phi=0,mask = []):

    if mask != []: # if there is a mask
        aux = f.copy()
        new_v = aux[mask].max() +1# value that the pixels outside the mask must assume
        aux[~mask] = new_v # all pixels out of the mask are set with f[mask].max()+1 (maximum value in the mask plus one)
        run_length = iarl(aux,theta,phi)*1.0

        # extract the lines and columns that corresponds to pixels outside the mask with value new_v
        run_length = run_length[:-1,:]# extract the row that represents the pixels out of the mask
        run_length = run_length[:,:np.argwhere(np.sum(run_length,axis=0)>0).max()+1]
        # in some cases, some different lengths of runs are added in this operation
        # on those cases, besides to extracted the row corresponds to new_v
        # it must be extracted the columns added in this operations (i.e. all null columns in the end of the matrix)

    else:
        run_length = iarl(f,theta,phi)
    C = run_length.sum()
    i,j = np.indices(run_length.shape)+1
    # compute descriptors
    ShrtREmph = ((run_length/(j*j)).sum())/C
    LngREmph = ((run_length*(j*j)).sum())/C
    GLevNonUni = (run_length.sum(axis=1)*run_length.sum(axis=1)).sum()/C
    RLNonUni = (run_length.sum(axis=0)*run_length.sum(axis=0)).sum()/C
    Fraction = C/((run_length*j).sum())
    return run_length,[RLNonUni,GLevNonUni,LngREmph,ShrtREmph,Fraction]

In [107]:
f = np.array([[[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]],
              [[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]],
              [[1,2,2,0,0,1],[0,0,1,2,2,1],[1,1,0,0,0,2],[1,1,1,2,2,2],[1,1,2,2,0,0]]], dtype=np.uint8)
#print 'input array \n',f
print f.shape[0]
print 
print 'THIS IS RLDESC'
print rldesc(f) # orientation straight up
# print 'rl: \n', rl
# print 'Run Length Descriptors\n',desc

3

THIS IS RLDESC
THIS IS EE.NDIM
3
offsets:  [[1 0 0]]
hoff:  [1]
woff:  [0]
loff:  [0]
h:  [[[1]]

 [[2]]

 [[3]]] 
w:  [[[0]
  [1]
  [2]
  [3]
  [4]]] 
l:  [[[0 1 2 3 4 5]]]
h:  [[[ 1]]

 [[ 2]]

 [[90]]] 
w:  [[[0]
  [1]
  [2]
  [3]
  [4]]] 
l:  [[[0 1 2 3 4 5]]]
h*W:
[[[  5]]

 [[ 10]]

 [[450]]]
w*L:
[[[ 0  3  6  9 12 15]]]
nonzero_nodes from iabelflat_unionfind [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89]


IndexError: index 89 is out of bounds for axis 0 with size 15